---
title: Data Cleaning
---

Now that we have gathered a reasonable amount of textual and record data, we can begin the data cleaning process. Our ultimate goal is to do statistical modeling wtih our data, so we need to clean the data. Generally, we will follow the principles of tidy data [@tidydata] in cleaning our data.

# Cleaning Text Data

To begin, we will clean our textual data by parsing out the text from the JSON and HTML objects that were returned from the Reddit and Wikipedia APIs respectively.

## Reddit Data

Recall that the Reddit data was returned as a JSON. We retrieved 10,000 text posts for each of three different text files. Our goal is to turn each of these JSON files into an individual dataframe. From there, we can transform the data into a Bag of Words, Document Term Matrix, or any other helpful format.

We will use `pandas` and `json` to parse this data into a desired output. First let's read in the data.

In [12]:
import pandas as pd
import json

with open("../data/reddit_psychosis_data.json") as f:
    reddit_psychosis = json.load(f)
with open('../data/reddit_cannabis_data.json') as f:
    reddit_cannabis = json.load(f)
with open("../data/reddit_schizophrenia_data.json") as f:
    reddit_schizophrenia = json.load(f)

From the data pull, we know that each of these JSON files is has 100 elements, each with 100 posts.

Let's look at the structure of one element to identify how we can extract the title and text information.


In [15]:
reddit_psychosis['0']

{'kind': 'Listing',
 'data': {'after': 't3_15z9mid',
  'dist': 100,
  'modhash': '',
  'geo_filter': '',
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'Psychosis',
     'selftext': '3 years post-psychosis in recovery some days can be a complete nightmare can be a nightmare but if u hold on long enough every time you get knocked down you grow stronger until you find your breakthrough moment &lt;3 trust me friends DM me for support ❤️',
     'author_fullname': 't2_8e0dojvz',
     'saved': False,
     'mod_reason_title': None,
     'gilded': 0,
     'clicked': False,
     'title': 'first time smiling on camera in... 3 years!',
     'link_flair_richtext': [],
     'subreddit_name_prefixed': 'r/Psychosis',
     'hidden': False,
     'pwls': None,
     'link_flair_css_class': None,
     'downs': 0,
     'thumbnail_height': 140,
     'top_awarded_type': None,
     'hide_score': False,
     'name': 't3_ybi1ao',
     'quarantine': False,
     'link_flair_

From here, we see that the posts are within the parameter `children`.

In [18]:
len(reddit_psychosis['0']['data']['children'])

100

It looks like these are the 100 posts we're looking for. Now we'll extract the title and text from each of these children elements.

In [34]:
reddit_psychosis['0']['data']['children'][1]['data']

{'approved_at_utc': None,
 'subreddit': 'Psychosis',
 'selftext': '',
 'author_fullname': 't2_im5nzt97',
 'saved': False,
 'mod_reason_title': None,
 'gilded': 0,
 'clicked': False,
 'title': 'I quit my meds lmfao',
 'link_flair_richtext': [],
 'subreddit_name_prefixed': 'r/Psychosis',
 'hidden': False,
 'pwls': None,
 'link_flair_css_class': None,
 'downs': 0,
 'thumbnail_height': 99,
 'top_awarded_type': None,
 'hide_score': False,
 'name': 't3_15q9gxw',
 'quarantine': False,
 'link_flair_text_color': 'dark',
 'upvote_ratio': 0.97,
 'author_flair_background_color': None,
 'subreddit_type': 'public',
 'ups': 380,
 'total_awards_received': 0,
 'media_embed': {},
 'thumbnail_width': 140,
 'author_flair_template_id': '40dc1c44-00e9-11e7-8565-0e172963bac8',
 'is_original_content': False,
 'user_reports': [],
 'secure_media': None,
 'is_reddit_media_domain': True,
 'is_meta': False,
 'category': None,
 'secure_media_embed': {},
 'link_flair_text': None,
 'can_mod_post': False,
 'score': 38

In [24]:
text = reddit_psychosis['0']['data']['children'][1]['data']['selftext']
title = reddit_psychosis['0']['data']['children'][0]['data']['title']

Since we have discovered the structure of this data, we can extract the text info for all of the posts. Let's loop through all three files to get the data in a data frame.

First, we can define function to loop through each of our JSON files.

In [52]:
def parse_reddit_json(reddit_json):
    text_list = []
    title_list = []
    subreddit_list = []
    for i in range(0, 100):
        index = str(i)
        for j in range(0, 100):
            text_list.append(reddit_json[index]['data']['children'][j]['data']['selftext'])
            title_list.append(reddit_json[index]['data']['children'][j]['data']['title'])
            subreddit_list.append(reddit_json[index]['data']['children'][j]['data']['subreddit'])

    return text_list, title_list, subreddit_list

Now, the function parses each of the JSON files and outputs a list of the title of each post and the text contents of each post. All posts have titles, but not all posts have additional text.

In [53]:
psy_text, psy_title, psy_sub = parse_reddit_json(reddit_psychosis)
schiz_text, schiz_title, schiz_sub = parse_reddit_json(reddit_schizophrenia)
cannabis_text, cannabis_title, cannabis_sub = parse_reddit_json(reddit_cannabis)

Now that we have these lists, we can combine them into a `pandas` dataframe where each row is one post on Reddit.

In [60]:
text = psy_text + schiz_text + cannabis_text
title = psy_title + schiz_title + cannabis_title
sub = psy_sub + schiz_sub + cannabis_sub

reddit_df = pd.DataFrame({'text': text, 'title': title, 'subreddit': sub})
reddit_df.head()

,text,title,subreddit
0,3 years post-psychosis in recovery some days c...,first time smiling on camera in... 3 years!,Psychosis
1,,I quit my meds lmfao,Psychosis
2,,I hate it here,Psychosis
3,,art by me. I thought it kinda visualized how I...,Psychosis
4,,But I’m still god and this is neither a joke a...,Psychosis


Now we have a data frame of labeled text objects that will be easy to work with for modeling.

## Wikipedia Data

Next, let's clean up our Wikipedia data. The Wikipedia API returned a complex nested R object. We already extracted the HTML from this R object and stored it in a csv, but we really want the main text of each webpage. 

We will use `rvest` to "harvest" the data from each HTML and store all of the information in a tibble.

In [5]:
library(rvest)

load("../data/wikipedia_scrape.Rdata")
wiki_data %>% names()

[1] "title" "text"  "link"

The HTML data is stored in the text column of the data frame. Let's take the first element to parse out the text from each webpage.

In [26]:
first <- wiki_data$text[1]
first %>% 
    read_html() %>%
    html_element("body") %>% 
    html_element("div") %>% 
    html_elements("p") %>% 
    html_text() %>% 
    head()


Throughout exploring the structure of the HTML file in Wikipedia, we see that there is a simple way to get all paragraph text from each page. We just need to pull all the text from the `<p>` tags on each page.

Now, we will loop through the tibble to convert the HTML string into a plain text string representing the paragraphs on the Wikipedia page.

In [45]:
text_column <- list()
for(i in 1:nrow(wiki_data)) {
    html <- wiki_data$text[i]
    text_list <- html %>%
        read_html() %>%
        html_element("body") %>%
        html_elements("p") %>%
        html_text()
    text_list <- paste0(text_list, collapse = " ")
    text_column <- append(text_column, text_list)
}

Now we have a list of all the text from each of these HTML files. Let's add this as a new column to our tibble and get rid of the huge HTML strings to decrease our memory footprint.

In [55]:
#| message: false
library(tidyverse)

wiki_data <- wiki_data %>%
    select(-text) %>%
    tibble::add_column(raw_text = text_column)

In [57]:
wiki_data %>% head()

title                         link
1 Long-term effects of cannabis main
2 (C6)-CP 47,497                link
3 (C9)-CP 47,497                link
4 (C9)-CP 47,497                link
5 (C9)-CP 47,497                link
6 (C9)-CP 47,497                link
  raw_text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

Now we have our text data from Wikipedia in a data frame with labels representing whether this was a forward or back link for the page titled "Long-term effects of cannabis."

# Record Data